In [9]:
import os
import imageio
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
import cv2
from keras import backend as K
from keras.models import load_model

from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes,detect_video
from yad2k.models.keras_yolo import yolo_head, yolo_eval

In [10]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
yolo_model = load_model("model_data/yolo.h5")
# yolo_model.summary()

E:\python\lib\site-packages\keras\engine\saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [11]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

model_image_size = yolo_model.layers[0].input_shape[1:3] # 608,608

In [15]:
video_name ='test_video1.wmv'
image_path = os.path.join('videos', video_name)
output_path = os.path.join('out', video_name)

In [16]:
def detect_video(image_path, output_path, model_image_size, yolo_model, class_names, yolo_outputs): 
    from keras import backend as K
    input_image_shape = K.placeholder(shape=(2, ))
    sess = K.get_session()
    
    video_in = cv2.VideoCapture(image_path)
    #width = video_in.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)   # float
    #height = video_in.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT) # float
    width, height = int(video_in.get(3)), int(video_in.get(4))
    FPS = video_in.get(5)
    
    video_out = cv2.VideoWriter()
    video_out.open(output_path, cv2.VideoWriter_fourcc(*'DIVX'), FPS, (width, height))
    
    width = np.array(width, dtype=float)
    height = np.array(height, dtype=float)
    image_shape = (height, width)
    
    while video_in.isOpened():
        ret, data = video_in.read()
        if ret==False:
            break
        video_array = cv2.cvtColor(data,cv2.COLOR_BGR2RGB)
        image = Image.fromarray(video_array,mode='RGB')
        resized_image = image.resize(tuple(reversed(model_image_size)), Image.BICUBIC)
        image_data = np.array(resized_image, dtype='float32')
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)   # Add batch dimension.
        
        boxes, scores, classes = yolo_eval(yolo_outputs, image_shape)
        out_boxes, out_scores, out_classes = sess.run([boxes, scores, classes],
                                                      feed_dict={yolo_model.input: image_data,
                                                                 input_image_shape: [image.size[1], image.size[0]],
                                                                 K.learning_phase(): 0})
        colors = generate_colors(class_names)
        draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
        video_out.write(cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR))
        
    sess.close()
    video_in.release()
    video_out.release()
    print("detect Done")

In [17]:
detect_video(image_path, output_path, model_image_size, yolo_model, class_names, yolo_outputs)

car 0.60 (108, 81) (136, 95)
car 0.63 (108, 81) (135, 96)
car 0.65 (115, 98) (157, 111)
car 0.63 (108, 81) (135, 96)
car 0.65 (115, 98) (157, 111)
car 0.62 (116, 98) (156, 111)
car 0.66 (108, 81) (135, 96)
car 0.63 (115, 98) (157, 111)
car 0.67 (108, 81) (135, 96)
car 0.62 (116, 98) (157, 111)
car 0.63 (109, 81) (135, 96)
car 0.63 (116, 98) (157, 111)
car 0.65 (108, 81) (135, 96)
car 0.64 (109, 81) (135, 96)
car 0.64 (115, 98) (156, 111)
car 0.63 (109, 81) (135, 96)
car 0.65 (116, 98) (156, 111)
car 0.64 (109, 81) (135, 96)
car 0.65 (116, 98) (156, 111)
car 0.63 (109, 81) (135, 96)
car 0.64 (116, 98) (156, 111)
car 0.64 (116, 98) (156, 111)
car 0.64 (116, 98) (156, 111)
car 0.64 (116, 98) (157, 111)
car 0.63 (116, 99) (157, 111)
car 0.63 (116, 99) (157, 110)
car 0.62 (117, 99) (157, 110)
car 0.62 (117, 99) (157, 110)
car 0.63 (116, 99) (157, 111)
car 0.63 (116, 99) (157, 111)
car 0.63 (116, 99) (157, 111)
car 0.63 (116, 99) (157, 111)
car 0.62 (108, 81) (135, 96)
car 0.65 (115, 98) (15